![oltp.png](./images/oltp.png "oltp.png")

In [0]:
%sql
USE SCHEMA `202507`;

CREATE OR REPLACE TABLE person_cdc_raw (                 -- no CDF property needed
  person_id     BIGINT,
  first_name    STRING,
  last_name     STRING,
  email         STRING,
  phone         STRING,
  date_of_birth DATE,
  street        STRING,
  city          STRING,
  state         STRING,
  postal_code   STRING,
  country       STRING,
  operation     STRING,            -- 'INSERT' | 'UPDATE' | 'DELETE'
  sequence_num  BIGINT,            -- strictly increasing per person_id
  record_ts     TIMESTAMP          -- event time (can differ from ingest time)
)
COMMENT 'Synthetic personal‑data feed with explicit CDC flags';

INSERT INTO person_cdc_raw
SELECT
  id + 1                                                  AS person_id,           -- 1 … 100
  element_at(array('Alice','Bob','Charlie','Diana','Eve',
                   'Frank','Grace','Hiro','Ivy','John'),
             INT(id % 10 + 1))                                 AS first_name,
  element_at(array('Smith','Johnson','Williams','Brown','Jones',
                   'Miller','Davis','Garcia','Rodriguez','Wilson'),
             INT(id % 10 + 1))                                   AS last_name,
  lower(element_at(array('Alice','Bob','Charlie','Diana','Eve',
                         'Frank','Grace','Hiro','Ivy','John'),
                   INT(id % 10 + 1)))
    || '.' ||
  lower(element_at(array('Smith','Johnson','Williams','Brown','Jones',
                         'Miller','Davis','Garcia','Rodriguez','Wilson'),
                   INT(id % 10 + 1)))
    || (id + 1) || '@example.com'                         AS email,
  concat('+1', lpad(cast(rand()*1e10 AS BIGINT), 10, '0')) AS phone,
  date_add('1990-01-01', cast(rand()*12500 AS INT))       AS date_of_birth,
  concat(100 + id, ' Main St')                            AS street,
  'Springfield'                                           AS city,
  'CA'                                                    AS state,
  lpad(cast(rand()*99999 AS INT), 5, '0')                 AS postal_code,
  'USA'                                                   AS country,
  'INSERT'                                                AS operation,
  1                                                       AS sequence_num,
  current_timestamp()                                     AS record_ts
FROM range(100);                                          -- generates 0‑99

In [0]:
ALTER TABLE `hub`.`202507`.`person_cdc_raw` SET TBLPROPERTIES (delta.enableChangeDataFeed = true)